# Experiment LLM using HuggingFace

In [ ]:
!pip install transformers accelerate sentencepiece

### Prompt categories

In [ ]:
# Zero-shot category
def prompt_zero_shot(puzzle):
    return f"""Los het volgende taalkundige raadsel op. Het bevat drie aanwijzingen en het antwoord is één woord dat op alle drie van toepassing is. Geef het antwoord op de eerste regel en daarna een korte uitleg.

Vraag: {puzzle}. Wat is het?
Antwoord:"""

# One-shot category
def prompt_one_shot(puzzle):
    return f"""Los het volgende taalkundige raadsel op. Het bevat drie aanwijzingen en het antwoord is één woord dat op alle drie van toepassing is. Geef het antwoord op de eerste regel en daarna een korte uitleg. Eerst een voorbeeld, daarna een nieuw raadsel.

Voorbeeld:
Vraag: Het is een onderdeel van een schip, een bevestigingsmiddel, en een gymnastiekoefening. Wat is het?
Antwoord: Schroef.

Nu het raadsel:
Vraag: {puzzle}. Wat is het?
Antwoord:"""

# Three-shot category
def prompt_three_shot(puzzle):
    return f"""Los het volgende taalkundige raadsel op. Het bevat drie aanwijzingen en het antwoord is één woord dat op alle drie van toepassing is. Geef het antwoord op de eerste regel en daarna een korte uitleg. Eerst drie voorbeelden, dan een nieuw raadsel.

Voorbeeld 1:
Vraag: Het is een onderdeel van een schip, een bevestigingsmiddel, en een gymnastiekoefening. Wat is het?
Antwoord: Schroef.

Voorbeeld 2:
Vraag: Het is een winkelketen, een beroep, en een onderdeel van een schip. Wat is het?
Antwoord: Zeeman.

Voorbeeld 3:
Vraag: Het is seksuele anatomie, een beledigend woord, en een vrucht. Wat is het?
Antwoord: Eikel.

Raadsel:
Vraag: {puzzle}. Wat is het?
Antwoord:"""

### Define files

In [ ]:
# File used for experiment
infile = "test_puzzles.txt" # set infile

# File used to save results
outfile = "results_test.txt" # set infile

### Run experiment

In [ ]:
import ast
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


# Read file
def getData(file):
    with open(file, "r") as f:
        return f.readlines()

# Write file
def writeData(file, data):
    with open(file, "w") as f:
        for line in data:
            f.write(f"{line}\n")

# Load data
data = getData(infile)

# Setup model
model_name = "BramVanroy/fietje-2-chat" # set model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

results = []
correct = 0
for i, page in enumerate(data):
    load = ast.literal_eval(page)
    puzzle = load["prompt"]
    answer = load["answer"]

    # Set prompt category
    prompt = prompt_zero_shot(puzzle) # set shot category

    print(f"Processing: {i+1}/{len(data)}")

    # Generate output
    output = generator(prompt, max_new_tokens=50, do_sample=True, top_k=50, top_p=0.95)
    result = output[0]["generated_text"]

    # Evaluate result
    if answer.lower() in result.lower():
        correct += 1

    results.append(str({"prompt": prompt, "answer": answer, "result": result}))

print(f"Correct: {correct}/{len(data)}")

# Export results
writeData(outfile, results)